## Solid sequence conversion
[Reference](https://www.biostars.org/p/43855/)

AA, CC, GG, TT : 0 <br>
AC, CA, GT, TG : 1 <br>
AG, CT, GA, TC : 2 <br>
AT, CG, GC, TA : 3


In [55]:
import csv
import pandas as pd
import glob
import math
from pathlib import Path

In [63]:
hpc_codes_dir = "/Volumes/Huitian/Projects/T_Cell_ChIP/codes_hpc"

#--- Get list of sra files, retrive Run IDs
sra_dir = "/Volumes/Huitian/Projects/T_Cell_ChIP/202012_ChIP/1_SRA_Run_Table_simplified"
solid_file = sra_dir + "/2015_IMMUNITY_Martinez_simplified.csv"
solid_srr = pd.read_csv(solid_file)['Run'].tolist()
solid_layout = pd.read_csv(solid_file)['LibraryLayout'].tolist()

In [66]:
#--- Write Hpc script
out_file = "0_1_solid_to_fastq"
out_dir = hpc_codes_dir + "/" + out_file
Path(out_dir).mkdir(parents=True, exist_ok=True)

hpc_wkdir = "/gpfs/group/pipkin/hdiao/T_Cell_ChIP/0_fastq"
tool_dir = "/gpfs/group/pipkin/hdiao/T_Cell_ChIP/codes_hpc/0_1_solid_to_fastq/solid_to_fastq.py"
for i in range(0, len(solid_srr)):
    i_outname = "%s/%s-%s.sh"%(out_dir, out_file,i)
    i_srr = solid_srr[i]
    i_layout = solid_layout[i]
    with open(i_outname, "w") as fout:
        wfout = csv.writer(fout, delimiter="\t",lineterminator='\n')
        wfout.writerow(["#!/bin/bash"])
        wfout.writerow([])
        wfout.writerow(["module load python"])
        wfout.writerow(["cd %s"%hpc_wkdir])
        wfout.writerow([])
        wfout.writerow(["python %s %s_1.fastq"%(tool_dir, i_srr)])
        if i_layout == "PAIRED":
            wfout.writerow(["python %s %s_2.fastq"%(tool_dir, i_srr)])
        